In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
acme = pd.read_excel('Acme.xlsx')

In [3]:
acme.head()

,impression_id,click,cost,currently_insured,number_of_vehicles,number_of_drivers,marital_status,rank,policies sold
0,1,1,10,N,1,1,M,1,1
1,2,0,10,N,2,1,M,4,0
2,3,0,10,N,1,1,S,2,0
3,4,0,10,Y,2,1,M,5,0
4,5,0,10,Y,2,2,S,4,0


In [4]:
acme.describe()

,impression_id,click,cost,number_of_vehicles,number_of_drivers,rank,policies sold
count,10000.00000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,0.137400,10.0,1.608300,1.416900,3.378500,0.054500
std,2886.89568,0.344286,0.0,0.488155,0.493071,1.307214,0.227013
min,1.00000,0.000000,10.0,1.000000,1.000000,1.000000,0.000000
25%,2500.75000,0.000000,10.0,1.000000,1.000000,2.000000,0.000000
50%,5000.50000,0.000000,10.0,2.000000,1.000000,3.000000,0.000000
75%,7500.25000,0.000000,10.0,2.000000,2.000000,5.000000,0.000000
max,10000.00000,1.000000,10.0,2.000000,2.000000,5.000000,1.000000
